# COVID-19 Dashboard

This notebook uses Göttingen Campus COVID-19 publication data to build a simple dashboard.

Author: Andreas Lüschow

2021/07/09

-----

## Imports

In [1]:
# run imports
%run ../imports.ipynb

# import constants from constants notebook
%run ../constants.ipynb

# import methods from utils notebook
%run ../utils.ipynb

env: GOOGLE_APPLICATION_CREDENTIALS=../../../bigquery_credentials.json


## Custom Imports

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import numpy as np

from dash.dependencies import Output, Input
from jupyter_dash import JupyterDash

In [3]:
# run the following line when on Binder
# JupyterDash.infer_jupyter_proxy_config()

-----

## Load data

In [4]:
# data for line plot
df = load(f"{TABLE_FOLDER}{table_name(GOE_COVID)}.csv")
df["date"] = pd.to_datetime(df["date"]).dt.strftime('%Y-%m')
df = df.groupby(['date', "type"], as_index=False).size()

# data for treemap
df_treemap_orig = load(f"{TABLE_FOLDER}{table_name(GOE_CATEGORIES_FOR)}.csv")
df_covid = load(f"{TABLE_FOLDER}{table_name(GOE_COVID)}.csv")
df_covid["date"] = pd.to_datetime(df_covid["date"]).dt.strftime('%Y-%m')
df_covid = df_covid.groupby(["id", "title", 'date', "type"], as_index=False).size()

## Start Dash

In [5]:
external_stylesheets = [
    {"href": "style.css", "rel": "stylesheet"},
]

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app.title = "Göttingen Campus COVID-19 Publications"

In [6]:
# HTML layout
app.layout = html.Div(
    children=[
        html.Div(
            children=[
                html.H1(children="Göttingen Campus COVID-19 Publications", className="header-title"),
                html.P(
                    children="Analysis of COVID-19 publications from the Göttingen Campus",
                    className="header-description"
                ),
            ],
            className="header"
        ),
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(children="Type of Publication", className="menu-title"),
                        dcc.Dropdown(
                            id="type-filter",
                            options=[
                                {"label": type, "value": type}
                                for type in np.sort(df.type.unique())
                            ],
                            multi=True,
                            clearable=True,
                            className="dropdown",
                        ),
                    ],
                ),
                html.Div(
                    children=[
                        html.Div(
                            children="Date Range (MM/DD/YYYY)",
                            className="menu-title"
                            ),
                        dcc.DatePickerRange(
                            id="date-range",
                            min_date_allowed=df.date.min(),
                            max_date_allowed=df.date.max(),
                            start_date=df.date.min(),
                            end_date=df.date.max(),
                        ),
                    ]
                ),
            ],
            className="menu",
        ),
        html.Div(
            children=[
                html.Div(
                    children=dcc.Graph(
                        id="size-chart",
                        config={"displayModeBar": False},
                    ),
                    className="card",
                ),
                html.Div(
                    children=dcc.Graph(
                        id="treemap-chart",
                        config={"displayModeBar": False},
                    ),
                    className="card",
                ),
            ],
            className="wrapper",
        ),
    ]
)

In [7]:
# callback functions
@app.callback(
    [Output("size-chart", "figure"), Output("treemap-chart", "figure")],
    [
        Input("type-filter", "value"),
        Input("date-range", "start_date"),
        Input("date-range", "end_date"),
    ],
)
def update_charts(type, start_date, end_date):
    if not type:
        # consider all facets
        type = list(df['type'].unique())
    if not isinstance(type, list):
        type = list(type)

    # line plot
    mask = (
        (df.type.isin(type))
        & (df.date >= start_date)
        & (df.date <= end_date)
    )
    filtered_data = df.loc[mask, :]
    filtered_data.drop(["type"], axis=1, inplace=True)
    filtered_data = filtered_data.groupby(["date"])["size"].sum().reset_index()
    filtered_data = filtered_data.drop_duplicates()

    size_chart_figure = {
        "data": [
            {
                "x": filtered_data["date"],
                "y": filtered_data["size"],
                "type": "lines",
            },
        ],
        "layout": {
            "title": {
                # "text": "Publications per Month",
                "x": 0.05,
                "xanchor": "left",
            },
            "xaxis": {"fixedrange": True},
            "yaxis": {"fixedrange": True},
            "colorway": ["#17B897"],
        },
    }

    # treemap
    df_treemap = pd.merge(
        df_covid[["id", "title", "date", "type", "size"]],
        df_treemap_orig[["id", "for_v1_first_level", "for_v1_second_level"]],
        on=['id']
    )

    # add link to publication
    link_ref = '<a href="https://app.dimensions.ai/details/publication/{id}" target="_blank">{title}</a>'
    df_treemap['item_link'] = df_treemap.apply(
        lambda item: link_ref.format(id=item["id"], title=item["title"]), axis=1
    )

    mask = (
            (df_treemap.type.isin(type))
            & (df_treemap.date >= start_date)
            & (df_treemap.date <= end_date)
    )
    filtered_data = df_treemap.loc[mask, :]
    filtered_data.drop(["type"], axis=1, inplace=True)
    filtered_data.dropna(subset=["for_v1_second_level", "for_v1_first_level", "item_link"], inplace=True)
    filtered_data = filtered_data.drop_duplicates()

    fig = px.treemap(filtered_data,
                     path=[px.Constant("All categories"), "for_v1_first_level", "for_v1_second_level", "item_link"],
                     color="for_v1_first_level",
                     color_discrete_sequence=px.colors.qualitative.Pastel2,
                     color_discrete_map={
                         "(?)": "lightgrey"
                     }
                     )

    fig.update_traces(hoverinfo='skip', hovertemplate=None)

    return size_chart_figure, fig

## Run dashboard

In [8]:
# run dashboard inside Jupyter notebook
app.run_server(mode="inline")

# run dashboard in Browser
# app.run_server()